In [1]:
import read_dao
import numpy as np
#import statsmodels.api as sm
import lightcurves
import glob
import sys
import variables

In [1]:
import lightcurves
import variables
import numpy as np
import analysis_routines as ar
import optical


In [2]:

optical_dir = '/Volumes/Annie/CRRP/OpticalCatalogs/'

In [3]:
lcv='V16'
lcv_file = folder+'lcvs/optical/'+target+lcv+'.lcv'
U, B, V, R, I = lightcurves.read_optical_lcv(lcv_file)
var = variables.welch_stetson_indices(V[0], V[1], V[2])
print var
variables.robust_weighted_mean(V[0], V[1])

(509.13837952932909, 0.8313948383945754)


14.633497809325224

In [4]:
target = 'NGC5904'
folder = '/Volumes/Annie/CRRP/'+target+'/'
ids, xcat, ycat, ra, dec = optical.read_optical_fnl(optical_dir, target)
ar.make_mir_catalog(['I1', 'I2'], target, ids, data_dir=folder)
target = 'NGC6441'
folder = '/Volumes/Annie/CRRP/'+target+'/'
ids, xcat, ycat, ra, dec = optical.read_optical_fnl(optical_dir, target)
ar.make_mir_catalog(['I1', 'I2'], target, ids, data_dir=folder)
target = 'NGC6121'
folder = '/Volumes/Annie/CRRP/'+target+'/'
ids, xcat, ycat, ra, dec = optical.read_optical_fnl(optical_dir, target)
ar.make_mir_catalog(['I1', 'I2'], target, ids, data_dir=folder)

Reading optical catalog for NGC5904...
Finished reading optical catalog.


Reading optical catalog for NGC6441...
Finished reading optical catalog.


Reading optical catalog for NGC6121...
Finished reading optical catalog.


In [4]:
ind=66094
print phot_data['id'][ind]
all_mags = phot_data['psf_mag'][ind]
all_errs = phot_data['psf_err'][ind]
all_times = phot_data['mjd'][ind]
good_mags = all_mags[~np.isnan(all_mags)]
good_errs = all_errs[~np.isnan(all_mags)]
good_times = all_times[~np.isnan(all_mags)]
print good_mags
print good_errs

variables.robust_weighted_mean(good_mags, good_errs)

variables.welch_stetson_indices(good_mags, good_errs, good_times)

66095
[ 17.78   17.917  17.517  18.29   17.631  17.722  18.153  17.426  17.756
  17.912  17.63   17.677  17.474  17.531  17.957  17.335  18.135  17.656
  17.701  18.089  17.189  18.17   17.932  18.054  17.928  17.544  17.967
  17.625  17.918  17.647  17.742  18.23   18.06   18.241  17.775  18.214
  18.103  17.848  17.935  18.72   17.586  17.807  18.26   17.591  18.432
  17.196  17.628  18.302  17.905  17.997  17.322  18.457  18.891  17.527
  17.776  18.672  17.478  17.695  17.963]
[ 0.2923  0.3141  0.3467  0.522   0.2784  0.3297  0.4135  0.256   0.3205
  0.4336  0.3312  0.3345  0.3014  0.3515  0.335   0.2867  0.3896  0.2978
  0.2818  0.4247  0.2864  0.3029  0.3353  0.2917  0.3277  0.3222  0.3113
  0.2493  0.3468  0.3062  0.3179  0.2947  0.2601  0.426   0.2941  0.4319
  0.4492  0.3567  0.3315  0.5835  0.3094  0.2407  0.3692  0.2376  0.5584
  0.3163  0.4042  0.542   0.337   0.5364  0.307   0.416   0.6605  0.633
  0.3752  0.4319  0.3339  0.2985  1.2752]


(-0.083999560684748634, 0.8209637569492072)

# Search for variables in Clement catalog

Finds all possible matches within 3 arcsec and plots their raw lightcurve

In [3]:
variables.find_variables_by_coord_mosaic(optical_folder, target, working_folder=folder)

Reading optical catalog for NGC6402...
Finished reading optical catalog.
V3 - 5 matches
Writing to file...


IOError: [Errno 2] No such file or directory: 'mosaic_lcvs/V3.lcv'

# Make MIR catalog

In [ ]:
ids, raw_phot = read_dao.read_raw('optical_alf.raw')

id_num = np.zeros(len(ids), dtype=int)
x = np.zeros(len(ids), dtype=float)
y = np.zeros(len(ids), dtype=float)
mags = np.zeros((len(ids), (len(raw_phot[0])-7)/2), dtype=float)
errs = np.zeros((len(ids), (len(raw_phot[0])-7)/2), dtype=float)
avgs = np.zeros(len(ids), dtype=float)
avg_err = np.zeros(len(ids), dtype=float)

for ind, star in enumerate(ids):
    id_num[ind] = raw_phot[ind][0]
    x[ind] = raw_phot[ind][1]
    y[ind] = raw_phot[ind][2]
    mags[ind] = raw_phot[ind][5:-2:2]
    errs[ind] = raw_phot[ind][6:-2:2]

    good = mags[ind] < 90
    data = mags[ind][good]
    weights = 1/errs[ind][good]**2
    weights = weights[abs(data-np.mean(data)) < 2.5*np.std(data)]
    data = data[abs(data-np.mean(data)) < 2.5*np.std(data)]
    avgs[ind] = np.average(data, weights=weights)
    avg_err[ind] = np.sqrt(1/np.sum(weights))

#    sm.robust.scale.huber(mags[ind][good])

data = np.c_[id_num, x, y, avgs, avg_err]
np.savetxt('catalog.txt',data, fmt='%8i %9.3f %9.3f %6.3f %5.3f')